In [70]:
%load_ext autoreload
%autoreload 2

from kernel_lib import *
from matrix import Matrix
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
vocab = ["This", "is", "a", "sentence"]
vocab_size = len(vocab)

pos_enc_sequence_len = 10
token_dims = vocab_size

In [72]:
pos_encodings = Matrix(pos_enc_sequence_len, token_dims, np.float32, gpu=True)
pos_encodings.alloc_on_gpu()
gen_pos_encodings(pos_encodings.a_gpu,
                  np.int32(pos_encodings.num_rows),
                  np.int32(pos_encodings.num_cols),
                  block=(pos_encodings.num_cols, pos_encodings.num_rows, 1))
cuda.Context.synchronize()

print(f"{pos_encodings=}")

pos_encodings=[[ 1.00000000e+00  0.00000000e+00  9.99999975e-05  0.00000000e+00]
 [ 5.40302277e-01  8.41471016e-01  5.40302281e-05  8.41471046e-05]
 [-4.16146815e-01  9.09297466e-01 -4.16146831e-05  9.09297451e-05]
 [-9.89992499e-01  1.41120002e-01 -9.89992477e-05  1.41119999e-05]
 [-6.53643668e-01 -7.56802499e-01 -6.53643656e-05 -7.56802474e-05]
 [ 2.83662170e-01 -9.58924294e-01  2.83662175e-05 -9.58924284e-05]
 [ 9.60170269e-01 -2.79415488e-01  9.60170291e-05 -2.79415490e-05]
 [ 7.53902256e-01  6.56986594e-01  7.53902277e-05  6.56986595e-05]
 [-1.45500034e-01  9.89358246e-01 -1.45500035e-05  9.89358232e-05]
 [-9.11130250e-01  4.12118495e-01 -9.11130264e-05  4.12118497e-05]]


In [73]:
sentence = "This is a sentence"
sentence_toks = [0, 1, 2, 3] # Straight forward
word2tok = {"This" : 0, "is" : 1, "a" : 2, "sentence" : 3}

In [74]:
embeddings = Matrix(vocab_size, token_dims, np.float32, gpu=True)
embeddings.alloc_on_gpu()
embeddings.init_uniform_rand(math.sqrt(6.0 / (embeddings.num_rows + embeddings.num_cols)))

cuda.Context.synchronize()

print(f"{embeddings=}")


embeddings=[[ 0.41607213  0.72918266 -0.7984399   0.81226754]
 [ 0.736365   -0.09292436  0.28980532 -0.67561984]
 [-0.0515829   0.0228521   0.47834936 -0.35547632]
 [ 0.37067556 -0.24508198  0.31422633 -0.3602407 ]]


In [75]:
#TODO: Add a function that adds two matrices but has the ability to "scale" down matrices based on which one is bigger, etc
# Special "trimmed" matrix add...
kernel_code = """
// Assumes embedding matrix has been sized such that Dim(embedding_matrix) < Dim(pos_enc)
extern "C" __global__ void add_pos_enc_and_embed(float* embedding_matrix, float* pos_enc, float* output, int N) {
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  if (idx < N) {
    output[idx] = embedding_matrix[idx] + pos_enc[idx];
  }
}
"""

mod = SourceModule(kernel_code,
                   no_extern_c=True,
                   options=["-std=c++11",
                           "-Xcompiler",
                           "-fPIC"])

add_pos_enc_and_embed = mod.get_function("add_pos_enc_and_embed")

In [76]:
embeddings_w_pos = Matrix(embeddings.num_rows, embeddings.num_cols, np.float32, gpu=True)
embeddings_w_pos.alloc_on_gpu()
add_pos_enc_and_embed(embeddings.a_gpu,
                      pos_encodings.a_gpu,
                      embeddings_w_pos.a_gpu,
                      np.int32(embeddings.num_elements()),
                      block=(embeddings.num_elements(), 1, 1))
cuda.Context.synchronize()

print(f"{embeddings_w_pos=}")

embeddings_w_pos=[[ 1.4160721   0.72918266 -0.7983399   0.81226754]
 [ 1.2766674   0.74854666  0.28985935 -0.6755357 ]
 [-0.46772972  0.9321496   0.47830775 -0.3553854 ]
 [-0.61931694 -0.10396197  0.31412733 -0.36022657]]


In [77]:
test_a = Matrix(4,4,np.float32,gpu=True)
test_a.alloc_on_gpu()
test_a.init_incremental()

test_b = Matrix(4,4,np.float32,gpu=True)
test_b.alloc_on_gpu()
test_b.init_incremental()

test_c = test_a * test_b
print(test_c)

[[ 56.  62.  68.  74.]
 [152. 174. 196. 218.]
 [248. 286. 324. 362.]
 [344. 398. 452. 506.]]


In [78]:
print(f"Initially: {test_c=}")
test_c = test_c / 2
print(f"After scalar divide: {test_c=}")

Initially: test_c=[[ 56.  62.  68.  74.]
 [152. 174. 196. 218.]
 [248. 286. 324. 362.]
 [344. 398. 452. 506.]]
After scalar divide: test_c=[[ 28.  31.  34.  37.]
 [ 76.  87.  98. 109.]
 [124. 143. 162. 181.]
 [172. 199. 226. 253.]]


In [79]:
# Embedding matrix = [vocab_size, token_dims]
# Technically you can make swap the dimensions of this and it will still work
# One way requires a transpose, the other doesn't
# Weights: [3 * token_dims, token_dims]

#TODO: Make a weight transpose instead for learnings... Even though a bit less efficient...
# weights = Matrix(3 * embeddings.num_cols, embeddings.num_cols, np.float32, gpu=True)
# weights.alloc_on_gpu()
weights_t = Matrix(embeddings_w_pos.num_cols, 3 * embeddings_w_pos.num_cols, np.float32, gpu=True)
weights_t.alloc_on_gpu()

# QKV matrix = [vocab_size, 3 * token_dims]
QKV = Matrix(embeddings_w_pos.num_rows, weights_t.num_cols, np.float32, gpu=True)
QKV.alloc_on_gpu()

regular_matmul(embeddings_w_pos.a_gpu,
               weights_t.a_gpu,
               np.int32(QKV.num_rows),
               np.int32(QKV.num_cols),
               np.int32(embeddings_w_pos.num_cols),
               QKV.a_gpu,
               block=(QKV.num_cols, QKV.num_rows, 1))

b = Matrix(QKV.num_cols, 1, np.float32, gpu=True)
b.alloc_on_gpu()
# TODO: Fix the scale here..?
b.init_uniform_rand(math.sqrt(6.0 / (embeddings.num_rows + embeddings.num_cols)))

QKV_b = Matrix(QKV.num_rows, QKV.num_cols, np.float32, gpu=True)
QKV_b.alloc_on_gpu()

add_matrix_w_vector(QKV.a_gpu,
                    b.a_gpu,
                    np.int32(QKV.num_rows),
                    np.int32(QKV.num_cols),
                    QKV_b.a_gpu,
                    block=(QKV.num_cols, QKV.num_rows, 1))

Q = Matrix(QKV_b.num_rows, QKV_b.num_cols / 3, np.float32, gpu=True)
Q.set_gpu_matrix(QKV_b.a_gpu, stride=QKV_b.num_cols, start_idx=(0 * QKV_b.num_cols) / 3)

K = Matrix(QKV_b.num_rows, QKV_b.num_cols / 3, np.float32, gpu=True)
K.set_gpu_matrix(QKV_b.a_gpu, stride=QKV_b.num_cols, start_idx=(1 * QKV_b.num_cols) / 3)

V = Matrix(QKV_b.num_rows, QKV_b.num_cols / 3, np.float32, gpu=True)
V.set_gpu_matrix(QKV_b.a_gpu, stride=QKV_b.num_cols, start_idx=(2 * QKV_b.num_cols) / 3)

score = (Q * K) / math.sqrt(Q.num_cols)
score_scaled = score / math.sqrt(Q.num_cols)

In [80]:
test_a = Matrix(4,4,np.float32,gpu=True)
test_a.alloc_on_gpu()
test_a.init_incremental()

test_output = Matrix(4,1,np.float32,gpu=True)
test_output.alloc_on_gpu()
test_output.init_incremental()

print(f"{test_a=}")
print(f"Before: {test_output=}")

matrix_row_wise_add(test_a.a_gpu,
        np.int32(test_a.num_rows),
        np.int32(test_a.num_cols),
        test_output.a_gpu,
        block=(test_a.num_cols,test_a.num_rows,1),
        shared=test_a.num_elements() * test_a.dtype().nbytes)

print(f"After: {test_output=}")

test_a=[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
Before: test_output=[[0.]
 [1.]
 [2.]
 [3.]]
After: test_output=[[ 6.]
 [22.]
 [38.]
 [54.]]


In [ ]:
print(f"{score=}")
print(f"{score_scaled=}")

score_scaled_row_sum = Matrix(score_scaled.num_rows, 1, np.float32, gpu=True)
score_scaled_row_sum.alloc_on_gpu()

matrix_row_wise_add(score_scaled.a_gpu,
        np.int32(score_scaled.num_rows),
        np.int32(score_scaled.num_cols),
        score_scaled_row_sum.a_gpu,
        block=(score_scaled.num_cols, score_scaled.num_rows, 1),
        shared=score_scaled.num_elements() * score_scaled.dtype().nbytes)

score=[[248327.92 284970.56 321182.72 358128.88]
 [161808.03 185685.22 209282.33 233356.56]
 [266780.1  306150.56 345059.34 384753.84]
 [247824.56 284393.3  320532.38 357403.94]]
score_scaled=[[124163.96  142485.28  160591.36  179064.44 ]
 [ 80904.016  92842.61  104641.164 116678.28 ]
 [133390.05  153075.28  172529.67  192376.92 ]
 [123912.28  142196.66  160266.19  178701.97 ]]


In [82]:
print(f"{score_scaled_row_sum=}")

score_scaled_softmaxed = Matrix(score_scaled.num_rows, score_scaled.num_cols, np.float32, gpu=True)
score_scaled_softmaxed.alloc_on_gpu()

softmax(score_scaled.a_gpu,
        score_scaled_row_sum.a_gpu,
        np.int32(score_scaled.num_rows),
        np.int32(score_scaled.num_cols),
        score_scaled_softmaxed.a_gpu,
        block=(score_scaled_softmaxed.num_cols, score_scaled_softmaxed.num_rows, 1))

print(f"{score_scaled_softmaxed=}")

score_scaled_row_sum=[[606305.  ]
 [395066.06]
 [651371.9 ]
 [605077.1 ]]
matrix[idx]: inf vector[row]:606305.000000
matrix[idx]: inf vector[row]:606305.000000
matrix[idx]: inf vector[row]:606305.000000
matrix[idx]: inf vector[row]:606305.000000
matrix[idx]: inf vector[row]:395066.062500
matrix[idx]: inf vector[row]:395066.062500
matrix[idx]: inf vector[row]:395066.062500
matrix[idx]: inf vector[row]:395066.062500
matrix[idx]: inf vector[row]:651371.875000
matrix[idx]: inf vector[row]:651371.875000
matrix[idx]: inf vector[row]:651371.875000
matrix[idx]: inf vector[row]:651371.875000
matrix[idx]: inf vector[row]:605077.125000
matrix[idx]: inf vector[row]:605077.125000
matrix[idx]: inf vector[row]:605077.125000
matrix[idx]: inf vector[row]:605077.125000
score_scaled_softmaxed=[[151728.19  98841.55 162927.02 151421.06]
 [161808.03 185685.22 209282.33 233356.56]
 [266780.1  306150.56 345059.34 384753.84]
 [247824.56 284393.3  320532.38 357403.94]]
